In [34]:
file_name = '../01_deepdao_daos/1687365380_deepdao_platforms.csv'

In [35]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,deepdaoDaoId,name,platformTitle,platformId,website,mostRecentProposalDate,aum
0,67785e63-1fc6-498c-b5e0-575d4e549c54,Kusama Council,Substrate,15,NaN,2022-11-23T16:21:42.028Z,NaN
1,8bc4f20f-9757-4029-aedd-0c3470497b21,Kusama Democracy,Substrate,15,NaN,2022-11-22T22:03:24.020Z,6.717582e+06
2,edb484b7-141e-49fa-be32-5c64652cfe5b,Kusama Technical Committee,Substrate,15,NaN,2022-11-22T12:18:30.019Z,NaN
3,bb817d24-bc47-4705-9805-e77062fc463a,Index,Snapshot,11,https://snapshot.org/#/index-coop.eth,2023-06-19T18:06:53.000Z,7.061461e+06
4,4137cd30-33fa-42bc-8395-5aa8920005f5,Decentraland DAO,Aragon,4,https://client.aragon.org/#/0xf47917b108ca4b82...,2023-04-13T15:23:59.000Z,7.475261e+07


In [36]:
# get pages with platform title is Snapshot
df = df[df['platformTitle'] == 'Snapshot']
df.shape

(44, 7)

In [37]:
df.head()

,deepdaoDaoId,name,platformTitle,platformId,website,mostRecentProposalDate,aum
3,bb817d24-bc47-4705-9805-e77062fc463a,Index,Snapshot,11,https://snapshot.org/#/index-coop.eth,2023-06-19T18:06:53.000Z,7.061461e+06
5,b468b73f-db05-48bf-806e-b5409e355583,Decentraland,Snapshot,11,https://snapshot.org/#/snapshot.dcl.eth,2023-06-19T21:10:28.000Z,NaN
6,03226e70-d266-4688-ac57-38d7426ea5aa,maticdao,Snapshot,11,https://snapshot.org/#/51588.eth,2022-09-19T09:10:37.000Z,NaN
7,12502572-4dbf-4363-af8b-b83b9f07784c,Polygon,Snapshot,11,https://snapshot.org/#/revotu.eth,2021-12-15T02:44:24.000Z,NaN
8,281ef1e4-15b8-46de-b463-307bdc4beb4b,polgon,Snapshot,11,https://snapshot.org/#/zhudapang3.eth,2023-05-10T20:59:00.000Z,NaN


In [38]:
# epoch from a year ago
import datetime
import time
year_ago = datetime.datetime.now() - datetime.timedelta(days=365)
YEAR_AGO_EPOCH = int(time.mktime(year_ago.timetuple()))
YEAR_AGO_EPOCH

1655823308

In [39]:
def generate_query(space, first, skip):
    base = """
        query Proposals {
         proposals(where: {space: "%s", created_gte: %s}, first: %s, skip: %s, orderDirection: desc, orderBy: "created") {
            id
            ipfs
            author
            created
            network
            symbol
            type
            strategies {
              network
              params
            }
            validation {
              params
            }
            plugins
            title
            # body
            discussion
            choices
            start
            end
            quorum
            privacy
            snapshot
            state
            link
            app
            scores
            scores_by_strategy
            scores_state
            scores_total
            scores_updated
            votes
            flagged
          }
        }
    """
    return base % (space, YEAR_AGO_EPOCH, first, skip)

In [40]:
df.iloc[0].website

'https://snapshot.org/#/index-coop.eth'

In [41]:
# get the part before the final slash and put it into a snapshotId column, handle nulls
df['snapshotId'] = df['website'].apply(lambda x: x.split('/')[-1] if type(x) == str else None)
df.head()

,deepdaoDaoId,name,platformTitle,platformId,website,mostRecentProposalDate,aum,snapshotId
3,bb817d24-bc47-4705-9805-e77062fc463a,Index,Snapshot,11,https://snapshot.org/#/index-coop.eth,2023-06-19T18:06:53.000Z,7.061461e+06,index-coop.eth
5,b468b73f-db05-48bf-806e-b5409e355583,Decentraland,Snapshot,11,https://snapshot.org/#/snapshot.dcl.eth,2023-06-19T21:10:28.000Z,NaN,snapshot.dcl.eth
6,03226e70-d266-4688-ac57-38d7426ea5aa,maticdao,Snapshot,11,https://snapshot.org/#/51588.eth,2022-09-19T09:10:37.000Z,NaN,51588.eth
7,12502572-4dbf-4363-af8b-b83b9f07784c,Polygon,Snapshot,11,https://snapshot.org/#/revotu.eth,2021-12-15T02:44:24.000Z,NaN,revotu.eth
8,281ef1e4-15b8-46de-b463-307bdc4beb4b,polgon,Snapshot,11,https://snapshot.org/#/zhudapang3.eth,2023-05-10T20:59:00.000Z,NaN,zhudapang3.eth


In [42]:
# keep only rows with a snapshotId
df = df[df['snapshotId'].notnull()]
# reindex
df = df.reset_index(drop=True)
df.shape

(44, 8)

In [43]:
import requests
import json
import pandas as pd

URL = "https://hub.snapshot.org/graphql"


def get_all_proposals(snapshotId):
    all_proposals = []

    has_next_page = True
    skip = 0
    while has_next_page:
        print('.')

        query = generate_query(snapshotId, 1000, skip)
        r = requests.post(URL, json={'query': query})
        data = json.loads(r.text)

        proposals = data['data']['proposals']
        all_proposals += proposals

        if len(proposals) < 1000:
            has_next_page = False
        skip += 1000

    all_proposals_df = pd.DataFrame(all_proposals)

    # add snapshotId to the df
    all_proposals_df['snapshotId'] = snapshotId
    return all_proposals_df


sid = 'magicappstore.eth'

test_proposals = get_all_proposals(sid)
test_proposals.shape

.


(84, 29)

In [44]:
test_proposals.tail()

,id,ipfs,author,created,network,symbol,type,strategies,validation,plugins,...,link,app,scores,scores_by_strategy,scores_state,scores_total,scores_updated,votes,flagged,snapshotId
79,0x78af69810183e97bccb9e04133d715d956d86c4e7349...,bafkreieurhlygbw2p6pamfivn74ihxjumngzchl52zmd5...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683560851,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,cross-the-ages,"[1107, 55]","[[1107], [55]]",final,1162,1684905010,1162,False,magicappstore.eth
80,0x0a3fd11d0d2e360a4f9ba95fc45e13f42541209d0017...,bafkreidkhhynwutbpvcxl26e25u22csjqsamcwqjzyajr...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683560568,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,metawear-fashiondome,"[1049, 86]","[[1049], [86]]",final,1135,1684904723,1135,False,magicappstore.eth
81,0xca20b7331789416525ef422dace3ac621baedd672073...,bafkreie53m7vfz3lfiyk45nyqemz5prro765bnioa4vbr...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683559583,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,kryxivia-game,"[1076, 61]","[[1076], [61]]",final,1137,1684904989,1137,False,magicappstore.eth
82,0xc1b651e953a24b93ae1c48917f3fba2e14d8ea4dadf0...,bafkreiawdormlimrgbuy4syzz6tz6azob6ay2ynxbh2qd...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683546940,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,war-legends,"[1179, 62]","[[1179], [62]]",final,1241,1684904846,1241,False,magicappstore.eth
83,0x29a572ca57e9648c5d4d0715d895724ae5782d10276b...,bafkreidn4e422ccn4s2f6hifgf4kranxvrea55ubfhkzq...,0x17A4382bd5cAB0e3D452b9e8BF07C96BC5Ec817A,1683050996,56,Magic Store Vote,single-choice,"[{'network': '56', 'params': {'url': 'https://...",{'params': {}},{},...,https://snapshot.org/#/magicappstore.eth/propo...,validation-simulator,"[6969, 142]","[[6969], [142]]",pending,7111,1687358551,7111,False,magicappstore.eth


In [45]:
# get the current epoch and make a directory
import time
import datetime
now = datetime.datetime.now()
EPOCH = int(time.mktime(now.timetuple()))
EPOCH
# make a directory
import os
directory = f'./{EPOCH}'
if not os.path.exists(directory):
    os.makedirs(directory)

In [46]:
for row in df[:12].itertuples():
    sid = row.snapshotId  # this is the key to query the api
    dao_id = row.deepdaoDaoId    # this is the foreign key we use to link back to the DeepDAO data

    if os.path.exists(f'{directory}/{dao_id}.csv'):
        print(f'{directory}/{dao_id}.csv already exists, skipping')
        continue

    proposals = get_all_proposals(sid)

    # save w daoId
    proposals.to_csv(f'{directory}/{dao_id}.csv', index=False)
    
    # add random wait
    import random
    import time
    wait_time = random.randint(1, 10)
    print(f'waiting {wait_time} seconds')
    time.sleep(wait_time)


.
waiting 9 seconds
.
waiting 3 seconds
.
waiting 1 seconds
.
waiting 5 seconds
.
waiting 2 seconds
.
waiting 9 seconds
.
waiting 3 seconds
.
waiting 7 seconds
.
waiting 4 seconds
.
waiting 1 seconds
.
waiting 3 seconds
.
waiting 9 seconds
